<a href="https://colab.research.google.com/github/Naeima/Correlation-between-Female-Fertility-and-Employment-Status/blob/main/foo_KG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
!pip install rdflib
!pip install requests
!pip install SPARQLWrapper
!pip install rdflib spacy
!pip python -m spacy download en_core_web_sm


In [ ]:
# @title
import io
import os
import rdflib
from rdflib import URIRef, Literal, Graph, RDF, Namespace, plugin
from rdflib.plugins import sparql
from rdflib.plugins.sparql import prepareQuery
from rdflib.parser import Parser
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery
from SPARQLWrapper import SPARQLWrapper, JSON, N3
import requests
import json
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
# @title
import spacy
from rdflib import Graph

# Load the English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Your RDF graph loading remains the same
foo_KG = Graph(); foo_KG.parse("https://naeima.github.io/fooKG/index.ttl", format="ttl")

def formulate_sparql(question):
    # Use spaCy NLP to analyze the question
    doc = nlp(question)

    # Example: Extract named entities and nouns as potential subjects/objects
    subjects = [ent.text for ent in doc.ents] + [token.text for token in doc if token.pos_ == "NOUN"]

    # Here, you'd need a mapping from natural language terms to RDF predicates/your ontology
    # This example is overly simplistic and for demonstration only
    predicates = {"located": "dbo:location", "created": "dbo:creationDate"}

    # Formulate a basic SPARQL query based on identified components
    # Note: A real implementation would require more sophisticated mapping logic
    sparql_query = "SELECT ?LuinID  ?lat ?long  ?date WHERE { ?subject <http://w3id.org/def/foo#Liun>   ?LuinID; <http://www.w3.org/2003/01/geo/wgs84_pos#lat>  ?lat; <http://www.w3.org/2003/01/geo/wgs84_pos#long>  ?long; <http://www.w3.org/2006/time#DateTime>    ?date.} limit 100 "

    return sparql_query

# Example bot interaction
question = input("Enter your question: ")
sparql_query = formulate_sparql(question)
print("Generated SPARQL query:", sparql_query)

# Execute the SPARQL query
results = foo_KG.query(sparql_query)
# Convert the results to a pandas DataFrame
data = pd.DataFrame(results)

# Display the DataFrame
print(data)


Enter your question: Where is Liun?
Generated SPARQL query: SELECT ?LuinID  ?lat ?long  ?date WHERE { ?subject <http://w3id.org/def/foo#Liun>   ?LuinID; <http://www.w3.org/2003/01/geo/wgs84_pos#lat>  ?lat; <http://www.w3.org/2003/01/geo/wgs84_pos#long>  ?long; <http://www.w3.org/2006/time#DateTime>    ?date.} limit 100 
            0           1            2                    3
0   1046AG505  41.5063176   -124.29118  2007-07-29T00:02:04
1   1047AG505   41.506389    -124.2907  2007-07-29T01:01:19
2   1048AG505  41.5062585   -124.29043  2007-07-29T02:00:59
3   1049AG505  41.5065374   -124.29005  2007-07-29T03:01:30
4   1050AG505  41.5053196  -124.289406  2007-07-29T04:00:55
..        ...         ...          ...                  ...
95  1141AG505  41.4329395  -124.969246  2007-08-20T01:01:19
96  1142AG505  41.4338574  -124.968376  2007-08-20T02:00:55
97  1143AG505  41.4338255   -124.96836  2007-08-20T03:00:52
98  1144AG505  41.4311213   -124.96676  2007-08-20T04:01:16
99  1145AG505  41.

In [ ]:
# @title
import folium
from folium.plugins import MarkerCluster

# Sample data
data = {
    'ID': ['1046AG505', '1047AG505', '1048AG505', '1049AG505', '1050AG505'],
    'Latitude': [41.5063176, 41.506389, 41.5062585, 41.5065374, 41.5053196],
    'Longitude': [-124.29118, -124.2907, -124.29043, -124.29005, -124.289406],
    'Timestamp': ['2007-07-29T00:02:04', '2007-07-29T01:01:19', '2007-07-29T02:00:59', '2007-07-29T03:01:30', '2007-07-29T04:00:55']
}

# Initialize a map centered around the first location
m = folium.Map(location=[data['Latitude'][0], data['Longitude'][0]], zoom_start=13, tiles=None)

# Add the satellite tile layer with attribution
folium.TileLayer(
    tiles='https://{s}.googleapis.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    max_zoom=20,
    subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
    overlay=False,
    control=True,
).add_to(m)

# Use a marker cluster to manage the markers
marker_cluster = MarkerCluster().add_to(m)

# Add markers to the map with hover text as timestamps
for i in range(len(data['ID'])):
    folium.Marker(
        location=[data['Latitude'][i], data['Longitude'][i]],
        popup=f"<i>ID: {data['ID'][i]}</i>",
        tooltip=data['Timestamp'][i],
    ).add_to(marker_cluster)

# Display the map
m
